In [ ]:
import pandas as pd
import joblib
import warnings
from sklearn.metrics import mean_absolute_error
warnings.filterwarnings("ignore")

In [ ]:
import re
from scripts.target_scripts import (
    assign_features_ks_hours,
    prepare_shift_and_lags,
    prepare_target_set,
)


def create_datasets():
    """
    Функция создания датасета
    Проходимся по Target_Name (названия КС в формате "КС-15"), вложенный цикл по сдвигам дней (48, 72, 96),
    вложенный цикл по направлению давления (входное/выходное - "Pin"/"Pout")

    """

    #! DEBUG  измени перед релизом
    print("HI")
    for Target_Name in [ "КС-16"]:
        # for Target_Name in ["КС-15"]:
        for Hours in [48]:
            # for Hours in [48]:
            Target_List = ["КС-15", "КС-16", "КС-17", "КС-19"]
            for mode in ["Pin"]:
                try:
                    # for mode in ["Pin"]:
                    # t_set = prepare_dataset(Target_List, Target_Name, "2",hours= Hours) #* В папке upload должны будут лежать

                    try:
                        t_set = pd.read_excel(
                            "uploaded/"
                            + Target_Name
                            + "_"
                            + mode
                            + "_"
                            + str(Hours)
                            + "_h.xlsx"
                        )
                    except Exception as e:
                        raise Exception(e)

                    # ! DEBUG
                    # try:
                    #     weather_dfs = prepare_weather_dict_dataset()

                    # except Exception as e:
                    #     raise(Exception"Prepare weather dict error:\n"+e)

                    try:
                        result = prepare_target_set(t_set, [])  #! DEBUG
                    except Exception as e:
                        raise Exception("Prepare target set error:\n")

                    try:
                        total_result = prepare_shift_and_lags(result)
                    except Exception as e:
                        raise Exception("Prapare shift and lags error:\n")

                    try:
                        total_result_new = total_result.rename(
                            columns=lambda x: re.sub("[^A-Za-z0-9_]+", "_", x)
                        )
                        new_names = {
                            col: re.sub(r"[^A-Za-zА-Яа-я0-9_]+", "", col)
                            for col in total_result.columns
                        }
                        new_n_list = list(new_names.values())
                        # [LightGBM] Исправление одинаковых колонок.
                        new_names = {
                            col: (
                                f"{new_col}_{i}"
                                if new_col in new_n_list[:i]
                                else new_col
                            )
                            for i, (col, new_col) in enumerate(new_names.items())
                        }
                        total_result_new = total_result.rename(columns=new_names)
                    except Exception as e:
                        raise Exception("Rename columns error:\n")

                    try:
                        train_set_column_names = (
                            pd.read_parquet(
                                "data/test_set_КС-16_Pin_48_h.parquet"
                            )
                            .rename(
                                columns={
                                    f"{mode}_target_shift_{Hours}h_{Target_Name[-2:]}": f"{mode}_target_shift_{Hours}h_КС{Target_Name[-2:]}"
                                },
                            )
                            .columns
                        )
                    except Exception as e:
                        raise Exception(
                            "Error getting columns names from train dataset:\n"
                        )

                    try:
                        result_final = assign_features_ks_hours(
                            total_result_new,
                            train_set_column_names,
                            mode,
                            Target_Name[-2:],
                            Hours,
                        )
                    except Exception as e:
                        print(e)

                    try:
                        result_final.to_parquet(
                            "data/"
                            + Target_Name
                            + "_"
                            + mode
                            + "_"
                            + str(Hours)
                            + "_h_new.parquet",
                            index=None,
                        )
                    except Exception as e:
                        print(e)
                except Exception as e:
                    print(e)
                    continue

In [ ]:
Target_Name = "КС-16"
mode = "Pin"
Hours = "48h"

df = (
    pd.read_parquet("data/test_set_КС-16_Pin_48_h.parquet")
    .rename(
        columns={
            f"{mode}_target_shift_{Hours}h_{Target_Name[-2:]}": f"{mode}_target_shift_{Hours}h_КС{Target_Name[-2:]}"
        },
    )
    .columns
)

In [ ]:
create_datasets()

In [ ]:
import os
import pandas as pd

# Directory containing the Parquet files
input_directory = "./buffer"
output_directory = "./uploaded"

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Iterate over all files in the input directory
for filename in os.listdir(input_directory):
    print("hi")
    if filename.endswith(".parquet") and filename.startswith("КС-16") :
        
        # Full path to the input Parquet file
        parquet_file = os.path.join(input_directory, filename)

        # Read the Parquet file into a DataFrame
        df = pd.read_parquet(parquet_file)

        # Define the output Excel file name
        excel_file = os.path.join(
            output_directory, f"{os.path.splitext(filename)[0]}.xlsx"
        )

        # Write the DataFrame to an Excel file
        df.to_excel(excel_file, index=False)

        print(f"Converted {filename} to {os.path.basename(excel_file)}")

In [ ]:
os.listdir(input_directory)

In [ ]:
create_datasets()

In [ ]:
result

КС 16
Hours 48h
Pmode оба

In [ ]:
def predict(
    ks: str,
    pmode: str,
    hours: str,
    date: str,
):
    """
    Parameters:
    KC - номер КС (15, 16, 17, 19 и тд)
    Pmode - Pin/Pout
    Hours - 48h, 72h, 96h
    date - дата, дд.мм.гггг
    """
    # Load the model
    model_path = f"ModelsNewFix/{pmode}_lag_{hours}_КС-{ks}_LGBM_model.joblib"
    loaded_model = joblib.load(model_path)
    # Load training and test data
    train_path = f"data/train_set_КС-{ks}_{pmode}_{hours[:-1]}_h.parquet"
    test_path = f"data/test_set_КС-{ks}_{pmode}_{hours[:-1]}_h.parquet"
    train = pd.read_parquet(train_path).rename({f"{pmode}_target_shift_{hours}_{ks}" : f"{pmode}_target_shift_{hours}_КС{ks}"})
    test = pd.read_parquet(test_path).rename({f"{pmode}_target_shift_{hours}_{ks}" : f"{pmode}_target_shift_{hours}_КС{ks}"})
    # Filter test data by date
    test = test[test["DateTime"].dt.strftime("%d.%m.%Y") == date]
    if len(test) == 0:
        test = train[train["DateTime"].dt.strftime("%d.%m.%Y") == date]
        if len(test) == 0:
            raise Exception("Error")
    # Prepare training data
    X_train = train.drop(columns=["DateTime", f"{pmode}_target_shift_{hours}_КС{ks}"])
    y_train = train[f"{pmode}_target_shift_{hours}_КС{ks}"]
    # Fit the model
    loaded_model.fit(X_train, y_train)
    # Prepare test data
    X_test = test.drop(columns=["DateTime"])
    # Create date index for the predictions
    date_index = test["DateTime"].apply(
        lambda x: x + pd.Timedelta(hours=int(hours[:-1]))
    )
    date_index = pd.to_datetime(date_index, errors="coerce").dt.strftime(
        "%Y-%m-%d %H:%M"
    )
    # Make predictions
    result = loaded_model.predict(X_test)
    prediction = pd.concat(
        [
            date_index.reset_index(drop=True),
            pd.DataFrame(result).apply(lambda x: round(x, 3)),
        ],
        axis=1,
    ).rename(columns={0: "Result"})
    # Add additional fields
    # prediction["mode"] = pmode
    # prediction["station"] = ks
    # Save prediction to Excel
    return prediction

In [ ]:
create_datasets()

In [ ]:
pd.read_csv("inspect/Object.csv", delimiter=";")

In [ ]:
from datetime import timedelta, datetime

start_date = datetime.strptime('03.01.2021', '%d.%m.%Y')
end_date = datetime.strptime('21.01.2024', '%d.%m.%Y')
best_day = start_date
best_score = 10000
# Iterate over the range of dates
current_date = start_date
while current_date <= end_date:

        result = predict("16","Pin","48h",(current_date - timedelta(days=2)).strftime('%d.%m.%Y'))
        result["DateTime"] = pd.to_datetime(result["DateTime"])
        y_pred = result[result["DateTime"].dt.strftime("%d.%m.%Y") == current_date.strftime("%d.%m.%Y")]["Result"]
        # df[df["Data"].dt.strftime("%d.%m.%Y %H:%M") == (current_date + timedelta(hours=1)).strftime("%d.%m.%Y %H:%M")][df["Object"] == 1]["P_Plan"]
        y_true = df[df["Data"].dt.strftime("%d.%m.%Y") == current_date.strftime("%d.%m.%Y")][df["Object"] == 2]["P_Fact"]
        y_true = df[df["Data"].dt.strftime("%d.%m.%Y") == current_date.strftime("%d.%m.%Y")][df["Object"] == 2]["P_Plan"]
        mae_result = mean_absolute_error(y_true, y_pred)
        if mae_result < best_score:
            best_score = mae_result
            best_day = current_date
        current_date += timedelta(days=1)
        # mae_result = mean_absolute_error(y_true, y_pred)
print({"mae": mae_result, "model": y_pred, "fact": y_true})


In [ ]:
best_score, best_day 

In [ ]:
import plotly #for plotting
from plotly import graph_objects as go
from plotly import express as px
from plotly.subplots import make_subplots




In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=roll_list["time"], y=roll_list['STAND2_RGH_SRV'], name="Позиция сервоклапана правой капсулы, %"))

fig.add_trace(go.Scatter(x=roll_list["time"], y=roll_list['STAND2_RGH_SRV_REF'], name="Задание на сервоклапан правой капсулы, %"))

fig.add_trace(go.Scatter(x=roll_list["time"], y=roll_list['STAND2_RGH_SRV_REF'], name="Задание на сервоклапан правой капсулы, %"))

fig.update_layout(

    title=dict(text="MAE Pin model= plan=", font=dict(size=18), automargin=True, yref='paper')

)

fig.show()
 

In [ ]:
def get_series_results_for_date(object_id:str,ks:str, pmode:str, hours:str, date_str: str):
    """
    Params
    ks - номер КС (15, 16, 17, 19 и тд)
    pmode - Pin/Pout
    hours - 48h, 72h, 96h
    date_str - дата
    """
    #* Открываем заготовленный план-факт
    df = pd.read_csv("inspect/P_in_Plan_Fact.csv", delimiter=";")
    df["Data"] = pd.to_datetime(df["Data"])
    date = datetime.strptime(date_str, "%d.%m.%Y")
    result = predict(ks, pmode, hours, (date - timedelta(days=2)).strftime("%d.%m.%Y"))
    result["DateTime"] = pd.to_datetime(result["DateTime"])
    y_pred = result[
        result["DateTime"].dt.strftime("%d.%m.%Y") == date.strftime("%d.%m.%Y")
    ]["Result"]
    # df[df["Data"].dt.strftime("%d.%m.%Y %H:%M") == (current_date + timedelta(hours=1)).strftime("%d.%m.%Y %H:%M")][df["Object"] == 1]["P_Plan"]
    y_true = df[df["Data"].dt.strftime("%d.%m.%Y") == date.strftime("%d.%m.%Y")][
        df["Object"] == 1
    ]["P_Fact"]
    mae_result = mean_absolute_error(y_true, y_pred)
    return {"mae": mae_result, "model": y_pred, "fact": y_true}

In [ ]:
get_series_results_for_date("1","15", "Pin", "48h", "03.01.2021")

In [ ]:
def get_best_result(object_id:str,ks:str, pmode:str, hours:str, start_date: str, end_date: str):
    """
    Params
    ks - номер КС (15, 16, 17, 19 и тд)
    pmode - Pin/Pout
    hours - 48h, 72h, 96h
    start_date - начальная дата 03.01.2021
    end_date - конечная дата 21.01.2021
    """
    # start_date = datetime.strptime('03.01.2021', '%d.%m.%Y')
    # end_date = datetime.strptime('21.01.2021', '%d.%m.%Y')
    best_day = start_date
    best_score = 10000
    # Iterate over the range of dates
    try:
        current_date = start_date
        while current_date <= end_date:
            df = pd.read_csv("inspect/P_in_Plan_Fact.csv", delimiter=";")
            df["Data"] = pd.to_datetime(df["Data"])
            if mae_result < best_score:
                    best_score = mae_result
                    best_day = current_date
            current_date += timedelta(days=1)
    except Exception as e:
        print("Ошибка в значениях: ",e)
        print("Цикл завершен")
    return best_day, best_score
